# Pre-Processing

In [1]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import warnings
import pandas as pd
import numpy as np
import nltk
import re
import csv

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#https://raw.githubusercontent.com/SalmatulFarida/PPW/gh-pages/crawling_berita_antaranews.csv

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/SalmatulFarida/PPW/gh-pages/crawling_berita_antaranews.csv')
df

,Tanggal,Penulis,Judul,Artikel,Label
0,"Senin, 4 Desember 2023 20:01 WIB",NaN,Sembulungan dan Hinako resmi perkuat pertahana...,ANTARA - Kapal Angkatan L...,hukum
1,"Senin, 4 Desember 2023 19:57 WIB",NaN,Wamenkumham Eddy Hiariej jadi saksi dugaan men...,NaN,hukum
2,"Senin, 4 Desember 2023 19:54 WIB",Pewarta: Fianda Sjofjan RassatEditor: Tasrief ...,Eddy Hiariej ajukan praperadilan terkait penet...,Jakarta (ANTARA) - Wakil Menteri Hukum da...,hukum
3,"Senin, 4 Desember 2023 19:42 WIB",NaN,Kanwil BPN Sulsel serahkan 200 sertifikat tana...,ANTARA - Kantor Wilayah B...,hukum
4,"Senin, 4 Desember 2023 19:28 WIB",NaN,Perketat pengawasan penyelundupan BBL di Banda...,ANTARA - Kementerian Kela...,hukum
...,...,...,...,...,...
715,"Kamis, 16 November 2023 13:34 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,SM Entertainment gandeng produser Inggris bent...,Jakarta (ANTARA) - Agensi SM Entertainmen...,hiburan
716,"Kamis, 16 November 2023 12:17 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,"AMPERS&ONE, grup idola K-pop baru debut lewat ...",Jakarta (ANTARA) - Grup idola K-pop penda...,hiburan
717,"Kamis, 16 November 2023 12:15 WIB",NaN,Pemenang Piala Citra FFI 2023,Ajang penghargaanPiala Citra Festifal Film...,hiburan
718,"Kamis, 16 November 2023 11:08 WIB",Penerjemah: Fitra AshariEditor: Maria Rosari D...,Bradley Cooper habiskan waktu enam tahun dalam...,Jakarta (ANTARA) - Dalam mendalami perann...,hiburan


Jumlah Raw Data Awal

In [4]:
count_hukum = 0
count_lifestyle= 0
count_hiburan = 0

for value in df['Label']:
  if value == 'hukum':
    count_hukum+=1
  elif value == 'lifestyle':
    count_lifestyle+=1
  else:
    count_hiburan += 1

print(
f'''
Banyak data ekonomi = {count_hukum} data
Banyak data olahraga= {count_lifestyle} data
Banyak data politik = {count_hiburan} data

Total Data          = {len(df)} data
'''
)


Banyak data ekonomi = 240 data
Banyak data olahraga= 240 data
Banyak data politik = 240 data

Total Data          = 720 data



Cek Missing Value

In [5]:
df.isna().sum()

Tanggal     0
Penulis    80
Judul       0
Artikel    31
Label       0
dtype: int64

Menghapus Missing Value

In [6]:
df = df.dropna()
df

,Tanggal,Penulis,Judul,Artikel,Label
2,"Senin, 4 Desember 2023 19:54 WIB",Pewarta: Fianda Sjofjan RassatEditor: Tasrief ...,Eddy Hiariej ajukan praperadilan terkait penet...,Jakarta (ANTARA) - Wakil Menteri Hukum da...,hukum
5,"Senin, 4 Desember 2023 19:18 WIB",Pewarta: Melalusa Susthira KhalidaEditor: Guid...,Wakil Ketua DPR respons soal urgensi revisi UU...,Jakarta (ANTARA) - Wakil Ketua DPR RI Suf...,hukum
8,"Senin, 4 Desember 2023 18:14 WIB",Pewarta: Victorianus Sat PranyotoEditor: Guido...,Bahu Teman Pemkab Sleman beri bantuan hukum ba...,Sleman (ANTARA) - Pemerintah Kabupaten(Pe...,hukum
10,"Senin, 4 Desember 2023 18:03 WIB",Pewarta: Melalusa Susthira KhalidaEditor: Laod...,Dasco: DPR sepakat tunda revisi UU MK diparipu...,Jakarta (ANTARA) - Wakil Ketua DPR RI Suf...,hukum
11,"Senin, 4 Desember 2023 17:36 WIB",Pewarta: Laily RahmawatyEditor: Laode Masrafi ...,Polri usulkan Marthinus Hukom sebagai Kepala BNN,Jakarta (ANTARA) - Kepolisian Negara Repu...,hukum
...,...,...,...,...,...
713,"Kamis, 16 November 2023 15:55 WIB",Pewarta: Vinny Shoffa SalmaEditor: Siti Zulaik...,Nessie Judge suarakan gerakan penuh dukung Pal...,Jakarta (ANTARA) - Pembuat konten Nessie ...,hiburan
715,"Kamis, 16 November 2023 13:34 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,SM Entertainment gandeng produser Inggris bent...,Jakarta (ANTARA) - Agensi SM Entertainmen...,hiburan
716,"Kamis, 16 November 2023 12:17 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,"AMPERS&ONE, grup idola K-pop baru debut lewat ...",Jakarta (ANTARA) - Grup idola K-pop penda...,hiburan
718,"Kamis, 16 November 2023 11:08 WIB",Penerjemah: Fitra AshariEditor: Maria Rosari D...,Bradley Cooper habiskan waktu enam tahun dalam...,Jakarta (ANTARA) - Dalam mendalami perann...,hiburan


Menghitung data duplikat

In [7]:
df.duplicated().sum()

46

Hapus data duplikat

In [8]:
df = df.drop_duplicates()
df

,Tanggal,Penulis,Judul,Artikel,Label
2,"Senin, 4 Desember 2023 19:54 WIB",Pewarta: Fianda Sjofjan RassatEditor: Tasrief ...,Eddy Hiariej ajukan praperadilan terkait penet...,Jakarta (ANTARA) - Wakil Menteri Hukum da...,hukum
5,"Senin, 4 Desember 2023 19:18 WIB",Pewarta: Melalusa Susthira KhalidaEditor: Guid...,Wakil Ketua DPR respons soal urgensi revisi UU...,Jakarta (ANTARA) - Wakil Ketua DPR RI Suf...,hukum
8,"Senin, 4 Desember 2023 18:14 WIB",Pewarta: Victorianus Sat PranyotoEditor: Guido...,Bahu Teman Pemkab Sleman beri bantuan hukum ba...,Sleman (ANTARA) - Pemerintah Kabupaten(Pe...,hukum
10,"Senin, 4 Desember 2023 18:03 WIB",Pewarta: Melalusa Susthira KhalidaEditor: Laod...,Dasco: DPR sepakat tunda revisi UU MK diparipu...,Jakarta (ANTARA) - Wakil Ketua DPR RI Suf...,hukum
11,"Senin, 4 Desember 2023 17:36 WIB",Pewarta: Laily RahmawatyEditor: Laode Masrafi ...,Polri usulkan Marthinus Hukom sebagai Kepala BNN,Jakarta (ANTARA) - Kepolisian Negara Repu...,hukum
...,...,...,...,...,...
713,"Kamis, 16 November 2023 15:55 WIB",Pewarta: Vinny Shoffa SalmaEditor: Siti Zulaik...,Nessie Judge suarakan gerakan penuh dukung Pal...,Jakarta (ANTARA) - Pembuat konten Nessie ...,hiburan
715,"Kamis, 16 November 2023 13:34 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,SM Entertainment gandeng produser Inggris bent...,Jakarta (ANTARA) - Agensi SM Entertainmen...,hiburan
716,"Kamis, 16 November 2023 12:17 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,"AMPERS&ONE, grup idola K-pop baru debut lewat ...",Jakarta (ANTARA) - Grup idola K-pop penda...,hiburan
718,"Kamis, 16 November 2023 11:08 WIB",Penerjemah: Fitra AshariEditor: Maria Rosari D...,Bradley Cooper habiskan waktu enam tahun dalam...,Jakarta (ANTARA) - Dalam mendalami perann...,hiburan


Jumlah Raw Data Akhir

In [9]:
count_hukum = 0
count_lifestyle= 0
count_hiburan = 0

for value in df['Label']:
  if value == 'hukum':
    count_hukum+=1
  elif value == 'lifestyle':
    count_lifestyle+=1
  else:
    count_hiburan += 1

print(
f'''
Banyak data ekonomi = {count_hukum} data
Banyak data olahraga= {count_lifestyle} data
Banyak data politik = {count_hiburan} data

Total Data          = {len(df)} data
'''
)


Banyak data ekonomi = 172 data
Banyak data olahraga= 214 data
Banyak data politik = 208 data

Total Data          = 594 data



In [10]:
df

,Tanggal,Penulis,Judul,Artikel,Label
2,"Senin, 4 Desember 2023 19:54 WIB",Pewarta: Fianda Sjofjan RassatEditor: Tasrief ...,Eddy Hiariej ajukan praperadilan terkait penet...,Jakarta (ANTARA) - Wakil Menteri Hukum da...,hukum
5,"Senin, 4 Desember 2023 19:18 WIB",Pewarta: Melalusa Susthira KhalidaEditor: Guid...,Wakil Ketua DPR respons soal urgensi revisi UU...,Jakarta (ANTARA) - Wakil Ketua DPR RI Suf...,hukum
8,"Senin, 4 Desember 2023 18:14 WIB",Pewarta: Victorianus Sat PranyotoEditor: Guido...,Bahu Teman Pemkab Sleman beri bantuan hukum ba...,Sleman (ANTARA) - Pemerintah Kabupaten(Pe...,hukum
10,"Senin, 4 Desember 2023 18:03 WIB",Pewarta: Melalusa Susthira KhalidaEditor: Laod...,Dasco: DPR sepakat tunda revisi UU MK diparipu...,Jakarta (ANTARA) - Wakil Ketua DPR RI Suf...,hukum
11,"Senin, 4 Desember 2023 17:36 WIB",Pewarta: Laily RahmawatyEditor: Laode Masrafi ...,Polri usulkan Marthinus Hukom sebagai Kepala BNN,Jakarta (ANTARA) - Kepolisian Negara Repu...,hukum
...,...,...,...,...,...
713,"Kamis, 16 November 2023 15:55 WIB",Pewarta: Vinny Shoffa SalmaEditor: Siti Zulaik...,Nessie Judge suarakan gerakan penuh dukung Pal...,Jakarta (ANTARA) - Pembuat konten Nessie ...,hiburan
715,"Kamis, 16 November 2023 13:34 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,SM Entertainment gandeng produser Inggris bent...,Jakarta (ANTARA) - Agensi SM Entertainmen...,hiburan
716,"Kamis, 16 November 2023 12:17 WIB",Penerjemah: Lia Wanadriani SantosaEditor: Mari...,"AMPERS&ONE, grup idola K-pop baru debut lewat ...",Jakarta (ANTARA) - Grup idola K-pop penda...,hiburan
718,"Kamis, 16 November 2023 11:08 WIB",Penerjemah: Fitra AshariEditor: Maria Rosari D...,Bradley Cooper habiskan waktu enam tahun dalam...,Jakarta (ANTARA) - Dalam mendalami perann...,hiburan


Cleaning data

In [11]:
def cleaning(text):
  text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
  return text

df['data_clean'] = df['Artikel'].apply(cleaning)
df['data_clean']

2      Jakarta ANTARA  Wakil Menteri Hukum dan Hak As...
5      Jakarta ANTARA  Wakil Ketua DPR RI Sufmi Dasco...
8      Sleman ANTARA  Pemerintah KabupatenPemkab Slem...
10     Jakarta ANTARA  Wakil Ketua DPR RI Sufmi Dasco...
11     Jakarta ANTARA  Kepolisian Negara Republik Ind...
                             ...                        
713    Jakarta ANTARA  Pembuat konten Nessie Judge me...
715    Jakarta ANTARA  Agensi SM Entertainment baruba...
716    Jakarta ANTARA  Grup idola Kpop pendatang baru...
718    Jakarta ANTARA  Dalam mendalami perannya untuk...
719    Jakarta ANTARA  Sony Pictures secara resmi men...
Name: data_clean, Length: 594, dtype: object

Tokenizing

In [12]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

df['Tokenizing'] = df['data_clean'].apply(tokenizer)
df['Tokenizing']

2      [jakarta, antara, wakil, menteri, hukum, dan, ...
5      [jakarta, antara, wakil, ketua, dpr, ri, sufmi...
8      [sleman, antara, pemerintah, kabupatenpemkab, ...
10     [jakarta, antara, wakil, ketua, dpr, ri, sufmi...
11     [jakarta, antara, kepolisian, negara, republik...
                             ...                        
713    [jakarta, antara, pembuat, konten, nessie, jud...
715    [jakarta, antara, agensi, sm, entertainment, b...
716    [jakarta, antara, grup, idola, kpop, pendatang...
718    [jakarta, antara, dalam, mendalami, perannya, ...
719    [jakarta, antara, sony, pictures, secara, resm...
Name: Tokenizing, Length: 594, dtype: object

Stopword

In [13]:
#custome stopword

from nltk.corpus import stopwords

# Mengambil daftar stopword bahasa Indonesia dari NLTK
stopwords_indonesia = set(stopwords.words("indonesian"))

# Sekarang, Anda memiliki daftar stopword yang telah diperbarui
print(stopwords_indonesia)

{'dipastikan', 'setelah', 'tampaknya', 'dimisalkan', 'disinilah', 'sebelumnya', 'jelaskan', 'kalau', 'menggunakan', 'berkata', 'supaya', 'lima', 'sudahkah', 'kalaulah', 'hari', 'biasanya', 'menyangkut', 'pada', 'sebab', 'olehnya', 'bagaimanakah', 'sejauh', 'mengakhiri', 'mempergunakan', 'ucapnya', 'sayalah', 'tuturnya', 'saya', 'rupanya', 'tadinya', 'sewaktu', 'segalanya', 'bermula', 'besar', 'tadi', 'inginkan', 'sekarang', 'pantas', 'diberi', 'inginkah', 'masing-masing', 'kembali', 'begitupun', 'asal', 'mulanya', 'dituturkan', 'teringat', 'mampukah', 'memihak', 'tambahnya', 'diberikan', 'ialah', 'keduanya', 'tahun', 'setengah', 'tutur', 'kemudian', 'sesama', 'diantaranya', 'dari', 'awalnya', 'sejenak', 'terjadilah', 'meskipun', 'se', 'keinginan', 'sangatlah', 'bakalan', 'didapat', 'itulah', 'begitu', 'siap', 'benar', 'mendatangkan', 'diingat', 'mengatakan', 'mungkin', 'serta', 'juga', 'namun', 'kedua', 'berawal', 'perlunya', 'nanti', 'jelaslah', 'sekadar', 'ditunjuk', 'bolehlah', 'wak

In [14]:
# Tambahkan kata-kata stopword kustom Anda ke dalam set
custom_stopwords = {"bisa-bisanya", "sebisanya", "mungkin", "ini"}

# Gabungkan kedua set stopword
stopwords_indonesia.update(custom_stopwords)

# Sekarang, Anda memiliki daftar stopword yang telah diperbarui
print(stopwords_indonesia)

{'dipastikan', 'setelah', 'tampaknya', 'dimisalkan', 'disinilah', 'sebelumnya', 'jelaskan', 'kalau', 'menggunakan', 'berkata', 'supaya', 'lima', 'sudahkah', 'kalaulah', 'hari', 'biasanya', 'menyangkut', 'pada', 'sebab', 'olehnya', 'bagaimanakah', 'sejauh', 'mengakhiri', 'mempergunakan', 'ucapnya', 'sayalah', 'tuturnya', 'saya', 'rupanya', 'tadinya', 'sewaktu', 'segalanya', 'bermula', 'besar', 'tadi', 'inginkan', 'sekarang', 'pantas', 'diberi', 'inginkah', 'masing-masing', 'kembali', 'begitupun', 'asal', 'mulanya', 'dituturkan', 'teringat', 'mampukah', 'memihak', 'tambahnya', 'diberikan', 'ialah', 'keduanya', 'tahun', 'setengah', 'tutur', 'kemudian', 'sesama', 'diantaranya', 'dari', 'awalnya', 'sejenak', 'terjadilah', 'meskipun', 'se', 'keinginan', 'sangatlah', 'bakalan', 'didapat', 'itulah', 'begitu', 'siap', 'benar', 'mendatangkan', 'diingat', 'mengatakan', 'mungkin', 'serta', 'juga', 'namun', 'kedua', 'berawal', 'perlunya', 'nanti', 'jelaslah', 'sekadar', 'ditunjuk', 'bolehlah', 'wak

In [15]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

df['Stopword Removal'] = df['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
df['stopword'] = df['Stopword Removal'].apply(lambda x: ' '.join(x))
df['stopword']

2      jakarta wakil menteri hukum hak asasi manusia ...
5      jakarta wakil ketua dpr ri sufmi dasco ahmad m...
8      sleman pemerintah kabupatenpemkab sleman daera...
10     jakarta wakil ketua dpr ri sufmi dasco ahmad d...
11     jakarta kepolisian negara republik indonesia p...
                             ...                        
713    jakarta pembuat konten nessie judge membagikan...
715    jakarta agensi sm entertainment barubaru menan...
716    jakarta grup idola kpop pendatang agensi fnc e...
718    jakarta mendalami perannya film maestro aktor ...
719    jakarta sony pictures resmi menghadirkan trail...
Name: stopword, Length: 594, dtype: object